In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode

In [ ]:
%%capture
%run "17. Eliminacion de outliers por tiempos.ipynb"

In [3]:
# Crear columnas de CO por ítem.
dfs_Finales = f.Crear_Columnas_Cambio_Opinion(dfs_Finales)

In [4]:
# Crear columnas de CT por ítem.
dfs_Finales = f.Crear_Columnas_Cambio_Tiempo(dfs_Finales)

In [ ]:
import pandas as pd
from typing import Dict, List


def Verificar_Columnas_Cambio(
    Dic_Dfs: Dict[str, pd.DataFrame]
) -> None:

    """
    
    Verifica la existencia de columnas CO y CT para cada ítem
    y cada lado (Izq/Der) en los DataFrames de entrada.
    
    """

    Items_IP: List[int] = [
        3, 4, 5, 6, 7, 8, 9, 10, 11,
        16, 19, 20, 22, 23, 24, 25,
        27, 28, 29, 30
    ]
    Lados: List[str] = ['Izq', 'Der']

    for Nombre_Df, Df in Dic_Dfs.items():
        print("\n" + "-" * 60)
        print(f"DATAFRAME: {Nombre_Df}")
        print("-" * 60)
        for Num in Items_IP:
            for Lado in Lados:
                Col_CO = f"CO_Item_{Num}_{Lado}"
                Col_CT = f"CT_Item_{Num}_{Lado}"
                if Col_CO not in Df.columns:
                    print(f"❌ Falta columna '{Col_CO}'.")
                if Col_CT not in Df.columns:
                    print(f"❌ Falta columna '{Col_CT}'.")
        print("✅ Verificación de presencia de columnas CO y CT completada.")

def Verificar_Valores_Cambio(
    Diccionario_Dfs: Dict[str, pd.DataFrame]
) -> None:

    """
    
    Verifica que los valores de CO y CT coincidan con las diferencias
    esperadas, tratando NaN==NaN como iguales y mostrando ejemplos
    aleatorios de discrepancias.

    Ejemplo:
        >>> Verificar_Valores_Cambio(dfs_Finales)

    """

    Items_IP: List[int] = [
        3, 4, 5, 6, 7, 8, 9, 10, 11,
        16, 19, 20, 22, 23, 24, 25,
        27, 28, 29, 30
    ]
    Lados: List[str] = ['Izq', 'Der']

    for Nombre_Df, Df in Diccionario_Dfs.items():
        print("\n" + "=" * 60)
        print(f"VALIDACIÓN DE VALORES - {Nombre_Df}")
        print("=" * 60)

        for Num in Items_IP:
            Base_Resp = f"IP_Item_{Num}_Respuesta"
            Base_Time = f"IP_Item_{Num}_Tiempo"
            for Lado in Lados:
                Col_CO = f"CO_Item_{Num}_{Lado}"
                Col_CT = f"CT_Item_{Num}_{Lado}"
                Lado_Resp = f"IP_Item_{Num}_{Lado}_Respuesta"
                Lado_Time = f"IP_Item_{Num}_{Lado}_Tiempo"

                # Validación de CO.
                if {Col_CO, Lado_Resp, Base_Resp}.issubset(Df.columns):
                    Esperado = (
                        pd.to_numeric(Df[Lado_Resp], errors='coerce')
                        - pd.to_numeric(Df[Base_Resp], errors='coerce')
                    )
                    Real = Df[Col_CO]
                    Iguales = (
                        (Real.isna() & Esperado.isna())
                        | (Real == Esperado)
                    )
                    Mismatches = ~Iguales
                    Count = int(Mismatches.sum())
                    if Count:
                        print(f"❌ {Col_CO}: {Count} discrepancias.")
                        print("   Primeras 3 discrepancias:")
                        for idx in Df[Mismatches].index[:3]:
                            val_r = Real.at[idx]
                            val_e = Esperado.at[idx]
                            print(f"     - Fila {idx}: {val_r!r} != {val_e!r}")
                        # Ejemplos aleatorios.
                        Indices = Df[Mismatches].index
                        Aleatorios = (
                            pd.Series(Indices)
                            .sample(n=min(3, len(Indices)),
                                    random_state=42)
                        )
                        print("   Ejemplos aleatorios:")
                        for idx in Aleatorios:
                            val_r = Real.at[idx]
                            val_e = Esperado.at[idx]
                            print(f"     * Fila {idx}: {val_r!r} != {val_e!r}")
                    else:
                        print(f"✅ {Col_CO} OK.")

                # Validación de CT.
                if {Col_CT, Lado_Time, Base_Time}.issubset(Df.columns):
                    EsperadoT = (
                        pd.to_numeric(Df[Lado_Time], errors='coerce')
                        - pd.to_numeric(Df[Base_Time], errors='coerce')
                    )
                    RealT = Df[Col_CT]
                    IgualesT = (
                        (RealT.isna() & EsperadoT.isna())
                        | (RealT == EsperadoT)
                    )
                    MismatchesT = ~IgualesT
                    CountT = int(MismatchesT.sum())
                    if CountT:
                        print(f"❌ {Col_CT}: {CountT} discrepancias.")
                        print("   Primeras 3 discrepancias:")
                        for idx in Df[MismatchesT].index[:3]:
                            val_rt = RealT.at[idx]
                            val_et = EsperadoT.at[idx]
                            print(f"     - Fila {idx}: {val_rt!r} != {val_et!r}")
                        # Ejemplos aleatorios.
                        IndicesT = Df[MismatchesT].index
                        AleatoriosT = (
                            pd.Series(IndicesT)
                            .sample(n=min(3, len(IndicesT)),
                                    random_state=42)
                        )
                        print("   Ejemplos aleatorios:")
                        for idx in AleatoriosT:
                            val_rt = RealT.at[idx]
                            val_et = EsperadoT.at[idx]
                            print(f"     * Fila {idx}: {val_rt!r} != {val_et!r}")
                    else:
                        print(f"✅ {Col_CT} OK.")

def Control_Ejemplos_Aleatorios_Valores_Cambio(
    Diccionario_Dfs: Dict[str, pd.DataFrame],
    Numero_Ids: int = 3,
    Random_State: int = 42
) -> None:

    """
    
    Para cada DataFrame en Diccionario_Dfs, selecciona aleatoriamente
    IDs de participantes y verifica manualmente los valores de CO y CT,
    comparando la resta esperada contra lo almacenado en el DataFrame.
    
    Parámetros:
    - Diccionario_Dfs: Diccionario con nombres y DataFrames a controlar.
    - Numero_Ids: Cantidad de IDs aleatorios a muestrear.
    - Random_State: Semilla para reproducibilidad de la muestra.
    
    """

    # Lista de ítems IP definidos en Funciones.py.
    Items_IP: List[int] = [
        3, 4, 5, 6, 7, 8, 9, 10, 11,
        16, 19, 20, 22, 23, 24, 25,
        27, 28, 29, 30
    ]
    Lados: List[str] = ['Izq', 'Der']

    for Nombre_Df, Data_Frame in Diccionario_Dfs.items():
        print("\n" + "=" * 60)
        print(f"CONTROL MANUAL ALEATORIO - {Nombre_Df}")
        print("=" * 60)

        # Verificar existencia de columna 'ID'.
        if 'ID' not in Data_Frame.columns:
            print("❌ Columna 'ID' no encontrada.")
            continue

        # Muestreo de IDs.
        Lista_Ids = Data_Frame['ID'].dropna().unique().tolist()
        if not Lista_Ids:
            print("❌ No hay IDs para muestrear.")
            continue
        Ids_Muestra = pd.Series(Lista_Ids).sample(
            n = min(Numero_Ids, len(Lista_Ids)),
            random_state = Random_State
        ).tolist()

        # Para cada ID aleatorio, mostrar cálculos.
        for Id in Ids_Muestra:
            Fila = Data_Frame.loc[Data_Frame['ID'] == Id]
            if Fila.empty:
                print(f"⚠️ Sujeto {Id} no encontrado en el DataFrame.")
                continue

            print(f"\n📋 Sujeto ID: {Id}")
            for Num in Items_IP:
                for Lado in Lados:
                    # Columnas base y con candidato para tiempo.
                    Col_Base = f"IP_Item_{Num}_Tiempo"
                    Col_Lado = f"IP_Item_{Num}_{Lado}_Tiempo"
                    Col_CT = f"CT_Item_{Num}_{Lado}"

                    if {Col_Base, Col_Lado, Col_CT}.issubset(
                        Data_Frame.columns
                    ):
                        Valor_Base = pd.to_numeric(
                            Fila[Col_Base].iloc[0],
                            errors = 'coerce'
                        )
                        Valor_Lado = pd.to_numeric(
                            Fila[Col_Lado].iloc[0],
                            errors = 'coerce'
                        )
                        Valor_CT = pd.to_numeric(
                            Fila[Col_CT].iloc[0],
                            errors = 'coerce'
                        )
                        CT_Esperado = Valor_Lado - Valor_Base

                        print(
                            f"   - Item {Num}_{Lado}: "
                            f"CT en DF={Valor_CT!r}, "
                            f"CT esperado={CT_Esperado!r}"
                        )

            for Num in Items_IP:
                for Lado in Lados:
                    # Columnas base y con candidato para respuesta.
                    Col_BaseR = f"IP_Item_{Num}_Respuesta"
                    Col_LadoR = f"IP_Item_{Num}_{Lado}_Respuesta"
                    Col_CO = f"CO_Item_{Num}_{Lado}"

                    if {Col_BaseR, Col_LadoR, Col_CO}.issubset(
                        Data_Frame.columns
                    ):
                        Valor_BaseR = pd.to_numeric(
                            Fila[Col_BaseR].iloc[0],
                            errors = 'coerce'
                        )
                        Valor_LadoR = pd.to_numeric(
                            Fila[Col_LadoR].iloc[0],
                            errors = 'coerce'
                        )
                        Valor_CO = pd.to_numeric(
                            Fila[Col_CO].iloc[0],
                            errors = 'coerce'
                        )
                        CO_Esperado = Valor_LadoR - Valor_BaseR

                        print(
                            f"   - Item {Num}_{Lado}: "
                            f"CO en DF={Valor_CO!r}, "
                            f"CO esperado={CO_Esperado!r}"
                        )

def Main() -> None:

    """
    
    Ejecuta las verificaciones de columnas y valores de cambio
    de opinión (CO) y cambio de tiempo (CT) para notebook 18.
    
    """

    print("=" * 80)
    print("CONTROL: COLUMNAS DE CO Y CT")
    print("=" * 80)

    if 'dfs_Finales' not in globals():
        print("❌ 'dfs_Finales' no encontrado.")
        return

    Dic_Dfs: Dict[str, pd.DataFrame] = globals()['dfs_Finales']

    Verificar_Columnas_Cambio(Dic_Dfs)
    Verificar_Valores_Cambio(Dic_Dfs)
    Control_Ejemplos_Aleatorios_Valores_Cambio(Dic_Dfs)




if __name__ == "__main__":
    Main()


In [6]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/18. Columnas de CO y CT ({Nombre_df}).xlsx', index=False)